In [ ]:
from pydub import AudioSegment
from pydub.utils import mediainfo
import os

def segment_and_validate_audio(audio_path, output_dir, chunk_duration=5):
    """
    Segments an audio file into fixed-duration chunks, checks its format,
    and converts it to the correct format if necessary. Discards chunks less than the specified duration.

    Args:
        audio_path (str): Path to the input audio file.
        output_dir (str): Directory to save the audio chunks.
        chunk_duration (int): Duration of each chunk in seconds (default is 5 seconds).

    Returns:
        tuple: (bool, list) - True if the audio was successfully processed, False otherwise,
               and a list of processed audio chunks.
    """
    # Validate the output directory
    os.makedirs(output_dir, exist_ok=True)

    # Check audio format using mediainfo
    info = mediainfo(audio_path)
    is_correct_format = (
        info["channels"] == "1" and
        info["sample_rate"] == "16000" and
        info["sample_fmt"] == "s16"
    )

    # Load the audio file
    audio = AudioSegment.from_file(audio_path)

    # Convert audio to correct format if necessary
    if not is_correct_format:
        print("Audio format is incorrect. Converting to 16-bit, 16 kHz, mono.")
        audio = audio.set_channels(1).set_frame_rate(16000).set_sample_width(2)  # 2 bytes = 16 bits

    # Segment the audio into chunks
    chunk_length_ms = chunk_duration * 1000
    base_name = os.path.splitext(os.path.basename(audio_path))[0]  # Extract base name without extension
    audio_chunks = []

    for i, start in enumerate(range(0, len(audio), chunk_length_ms)):
        chunk = audio[start:start + chunk_length_ms]

        # Skip chunks less than the desired duration
        if len(chunk) < chunk_length_ms:
            continue

        chunk_name = os.path.join(output_dir, f"{base_name}.wav")
        chunk.export(chunk_name, format="wav")
        print(f"Exported: {chunk_name}")
        audio_chunks.append(chunk)

    print("Segmentation complete.")
    return True, audio_chunks


# Example usage
segment_and_validate_audio("/content/antevy_female_23_10106.wav", "output_chunks")


Audio format is incorrect. Converting to 16-bit, 16 kHz, mono.
Exported: output_chunks/antevy_female_23_10106.wav
Segmentation complete.


(True, [<pydub.audio_segment.AudioSegment at 0x7d4f222e7d30>])

In [ ]:
!pip install pydub

# Inspect

In [ ]:
import numpy as np
from scipy.io import wavfile
from scipy.signal import resample
def inspect_wav_file(file_path):
    # Read the audio file
    sample_rate, data = wavfile.read(file_path)

    # Determine the number of channels (1 for mono, 2 for stereo)
    if len(data.shape) == 1:
        num_channels = 1  # Mono
    else:
        num_channels = data.shape[1]

    # Determine bit depth by inspecting the dtype of the audio data
    bit_depth = data.dtype

    return sample_rate, num_channels, bit_depth


file_path = '/content/output_chunks/chunk_1.wav'
sample_rate, num_channels, bit_depth = inspect_wav_file(file_path)

print(f"Sample Rate: {sample_rate} Hz")
print(f"Number of Channels: {num_channels} (1 = Mono, 2 = Stereo)")
print(f"Bit Depth: {bit_depth}")


Sample Rate: 16000 Hz
Number of Channels: 1 (1 = Mono, 2 = Stereo)
Bit Depth: int16


# Create temp and delete folder

In [ ]:
import tempfile
import os
import shutil

# Create a temporary folder
temp_folder = tempfile.mkdtemp()
print(f"Temporary folder created at: {temp_folder}")

# Perform operations inside the temp folder
# For example, creating a file in the temp folder:
file_path = os.path.join(temp_folder, "example.txt")
with open(file_path, "w") as f:
    f.write("This is an example file.")

print(f"File created: {file_path}")

# Clean up the temporary folder
shutil.rmtree(temp_folder)
print(f"Temporary folder deleted: {temp_folder}")
